In [1]:
from _Setup import *

In [2]:
directory = '/Users/pin.lyu/Desktop/2024_Case_Comp_BC/Data'

train_data = pd.read_csv(os.path.join(directory, 'train.csv'))
test_data = pd.read_csv(os.path.join(directory, 'test.csv'))

/var/folders/0x/ry77ppt564g_7fms7vcmt7nw0000gn/T/ipykernel_31547/1011239765.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(os.path.join(directory, 'test.csv'))


##### DEFINE BACK/FORWARD IMPUTATION FUNCTIONS

In [3]:
# backward fill function
def backward_fill_by_patient(df, patient_id_col='patient'):
    # Create a copy of the original DataFrame
    df_imputed = df.copy()

    df_imputed.loc[:, df_imputed.columns != patient_id_col] = (
        df_imputed.groupby(patient_id_col)
        .bfill()
    )

    # If there are patients with no entries for a column, those columns will remain unchanged
    return df_imputed


def forward_fill_by_patient(df, patient_id_col='patient'):
    # Create a copy of the original DataFrame
    df_imputed = df.copy()

    df_imputed.loc[:, df_imputed.columns != patient_id_col] = (
        df_imputed.groupby(patient_id_col)
        .ffill()
    )
    
    # If there are patients with no entries for a column, those columns will remain unchanged
    return df_imputed

##### DATA TRANSFORMATION

In [4]:
# Identify columns with more than 90% missing values

missing_percentage = train_data.isnull().mean() * 100

cols_to_drop_for_missing_percentage = missing_percentage[missing_percentage > 90].index

# Drop the identified columns

train_truncated = train_data.drop(columns=cols_to_drop_for_missing_percentage)

In [5]:
#forward imputation

train_forward = forward_fill_by_patient(train_truncated)

train_forward.head()

,patient,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,Glucose,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,16426,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.03,1.0,0.0,1.0,-4.25,1.0,0.0
1,16426,2,78.0,99.5,35.25,108.5,84.5,69.0,12.0,106.0,57.03,1.0,0.0,1.0,-4.25,2.0,0.0
2,16426,3,80.0,99.0,36.20,113.0,87.0,73.0,12.0,106.0,57.03,1.0,0.0,1.0,-4.25,3.0,0.0
3,16426,4,79.0,100.0,36.50,112.0,83.0,68.0,12.0,96.0,57.03,1.0,0.0,1.0,-4.25,4.0,0.0
4,16426,5,73.0,100.0,36.50,115.0,80.0,64.0,11.0,96.0,57.03,1.0,0.0,1.0,-4.25,5.0,0.0


In [6]:
#backward imputation

pre_imputed_train = backward_fill_by_patient(train_forward)

pre_imputed_train.head()  

,patient,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,Glucose,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,16426,1,78.0,99.5,35.25,108.5,84.5,69.0,12.0,106.0,57.03,1.0,0.0,1.0,-4.25,1.0,0.0
1,16426,2,78.0,99.5,35.25,108.5,84.5,69.0,12.0,106.0,57.03,1.0,0.0,1.0,-4.25,2.0,0.0
2,16426,3,80.0,99.0,36.20,113.0,87.0,73.0,12.0,106.0,57.03,1.0,0.0,1.0,-4.25,3.0,0.0
3,16426,4,79.0,100.0,36.50,112.0,83.0,68.0,12.0,96.0,57.03,1.0,0.0,1.0,-4.25,4.0,0.0
4,16426,5,73.0,100.0,36.50,115.0,80.0,64.0,11.0,96.0,57.03,1.0,0.0,1.0,-4.25,5.0,0.0


In [7]:
# Calculate missing percentage in each column 

naPercentages = pd.DataFrame(pre_imputed_train.isna().sum() / pre_imputed_train.shape[0], columns=['Percentage'])

naPercentages.sort_values(by = ['Percentage'], ascending=False)           

,Percentage
Unit1,0.391502
Unit2,0.391502
DBP,0.166844
Glucose,0.026092
SBP,0.006932
Temp,0.004656
MAP,0.001887
Resp,0.001336
O2Sat,0.000301
HR,0.000130


In [8]:
# implemment MissForest to impute the rest

imputer = MissForest()

imputer_input = pre_imputed_train.drop('SepsisLabel', axis = 1)

imputed_data = imputer.fit_transform(imputer_input)

imputed_data['SepsisLabel'] = pre_imputed_train['SepsisLabel']

imputed_data.isna().sum()

100%|█████████████████████████████████████████████| 5/5 [01:39<00:00, 19.95s/it]


patient        0
time           0
HR             0
O2Sat          0
Temp           0
SBP            0
MAP            0
DBP            0
Resp           0
Glucose        0
Age            0
Gender         0
Unit1          0
Unit2          0
HospAdmTime    0
ICULOS         0
SepsisLabel    0
dtype: int64

**MissForest Imputation**

**Pros:**

1. _Flexible Modeling_: MissForest is a non-parametric method that uses Random Forests, allowing it to capture complex relationships between variables.

2. _Handles Mixed Data Types_: It can effectively handle both categorical and continuous variables without needing prior transformations.
No Need for Distributional Assumptions: MissForest does not require assumptions about the distribution of the data, making it robust in various scenarios.

3. _Good Performance_: In many empirical studies, MissForest has shown competitive performance in terms of imputation accuracy compared to other methods.

**Cons:**

1. _Computationally Intensive_: It can be slow, especially with large datasets or a high number of variables due to the iterative nature of Random Forest training.

2. _Black Box_: The method can be difficult to interpret since it uses ensemble learning, making it less transparent than simpler methods.


3. _Requires Sufficient Data_: MissForest may struggle with datasets that have extensive missingness or few available observations for certain groups.


**MICE Imputation**

**Pros:**

1. _Multiple Imputation Framework_: MICE creates multiple datasets to account for uncertainty in the imputations, providing more reliable estimates of variability.

2. _Customizable_: Users can specify different imputation models for different variables, allowing for tailored approaches based on variable characteristics.

3. _Wide Applicability_: MICE is widely used in various fields and supported by many statistical software packages, making it accessible for practitioners.

4. _Results Interpretation_: The method allows for straightforward interpretation of results, as the imputation models are generally simpler and more transparent.

**Cons:**

1. _Model Dependency_: The quality of imputations heavily relies on the chosen imputation models, which may not capture complex relationships in the data.

2. _Computationally Intensive_: Like MissForest, MICE can be computationally demanding, particularly with larger datasets and multiple iterations.
   
3. _Assumption of Missing Data Mechanism_: MICE typically assumes that data is missing at random (MAR), which may not always be valid.
   
4. _Potential for Bias_: If the assumptions of the imputation model are violated, MICE may introduce bias into the estimates.

**Summary**
In summary, the choice between MissForest and MICE often depends on the specific dataset and research objectives. MissForest is beneficial for complex, non-linear relationships and mixed data types, while MICE is advantageous when dealing with multiple imputation scenarios and requires greater interpretability. Each method has its strengths and weaknesses, and it may be valuable to compare results from both approaches when possible.

##### TRAIN / VALIDATION SPLIT

In [9]:
# Create variable to see if patient ever has sepsis

imputed_data['SepsisPatient'] = imputed_data.groupby('patient')['SepsisLabel'].transform(lambda x: 1 if (x == 1).any() else 0)

# Create create [patient, SepsisEver] pair to take stratified sample

patientsList = imputed_data.drop_duplicates(subset=['patient', 'SepsisPatient'])[['patient', 'SepsisPatient']]

X_sample = patientsList['patient']

y_sample =  patientsList['SepsisPatient']

# Take stratified sample based upon whether individual ever has Sepsis

patient_labels_train, patient_labels_validate, _, _ = train_test_split(X_sample, y_sample, test_size=0.2, stratify=y_sample)

# Reexpand Train Data

train_split_data = imputed_data[imputed_data['patient'].isin(patient_labels_train)].drop('SepsisPatient', axis = 1)

validate_split_data = imputed_data[imputed_data['patient'].isin(patient_labels_validate)].drop('SepsisPatient', axis = 1)

##### SMOTE TRAIN SET

In [10]:
from imblearn.over_sampling import ADASYN
from collections import Counter

In [11]:
X = train_split_data.drop('SepsisLabel', axis = 1)

y = train_split_data['SepsisLabel']

# Best parameters

sm = SMOTE(sampling_strategy=1, k_neighbors=4, random_state=8)

X_train_syn, y_train_syn = sm.fit_resample(X, y)

# Counting the number of instances before oversampling

counter = Counter(y)

print('Before', counter)

# Counting the number of instances after oversampling

counter = Counter(y_train_syn)

print('After', counter)

Before Counter({0.0: 852557, 1.0: 15690})
After Counter({0.0: 852557, 1.0: 852557})


In [12]:
# Counting the number of instances in each class before oversampling
#counter = Counter(y)
#print('Before', counter)

# Oversampling the train dataset using ADASYN
#ada = ADASYN(sampling_strategy=1, n_neighbors=10, random_state=272)
#X_train_syn, y_train_syn = ada.fit_resample(X, y)

# Counting the number of instances in each class after oversampling
#counter = Counter(y_train_syn)
#print('After', counter)

##### MODEL

XGboost 
https://www.youtube.com/watch?v=TyvYZ26alZs

In [13]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [14]:
#Creating an XGBoost classifier

model = xgb.XGBClassifier()

#Training the model on the training data

model.fit(X_train_syn, y_train_syn)

# Predicting on the training set

y_train_pred = model.predict(X_train_syn)

# Calculate training accuracy

train_accuracy = accuracy_score(y_train_syn, y_train_pred)
print(f"Training Accuracy: {train_accuracy}")

# Detailed classification report

print("Classification Report on Training Set:")
print(classification_report(y_train_syn, y_train_pred))

# Confusion Matrix

print("Confusion Matrix on Training Set:")
print(confusion_matrix(y_train_syn, y_train_pred))

Training Accuracy: 0.9628159759406116
Classification Report on Training Set:
              precision    recall  f1-score   support

         0.0       0.95      0.98      0.96    852557
         1.0       0.97      0.95      0.96    852557

    accuracy                           0.96   1705114
   macro avg       0.96      0.96      0.96   1705114
weighted avg       0.96      0.96      0.96   1705114

Confusion Matrix on Training Set:
[[831509  21048]
 [ 42355 810202]]


In [15]:
# Check performance on validation set

# Split features and target from validation set

X_val = validate_split_data.drop('SepsisLabel', axis=1)

y_val = validate_split_data['SepsisLabel']

# Predict on validation set

y_val_pred = model.predict(X_val)

# Calculate Validation accuracy

val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Validation Accuracy: {val_accuracy}")

# Detailed classification report

print("Classification Report on Validation Set:")

print(classification_report(y_val, y_val_pred))

# Confusion Matrix

print("Confusion Matrix on Validation Set:")

print(confusion_matrix(y_val, y_val_pred))

Validation Accuracy: 0.9491748124573767
Classification Report on Validation Set:
              precision    recall  f1-score   support

         0.0       0.99      0.96      0.97    216018
         1.0       0.10      0.23      0.14      3932

    accuracy                           0.95    219950
   macro avg       0.54      0.60      0.56    219950
weighted avg       0.97      0.95      0.96    219950

Confusion Matrix on Validation Set:
[[207857   8161]
 [  3018    914]]


**Overall Validation Accuracy:**

The validation accuracy is 96.73%, which initially appears high. However, this can be misleading in cases of class imbalance, as accuracy does not account for how well the model performs on each class individually.
Class Imbalance:

The dataset is heavily imbalanced, with a large majority of samples belonging to class 0.0 (non-Sepsis) and only a small fraction to class 1.0 (Sepsis). This imbalance can lead the model to prioritize predicting the majority class (0.0), resulting in low performance on the minority class (1.0).
Precision, Recall, and F1-Score:

Class 0.0 (Non-Sepsis):
Precision: 0.98 - This means that when the model predicts 0.0 (non-Sepsis), it is correct 98% of the time.
Recall: 0.98 - The model correctly identifies 98% of actual non-Sepsis cases.
F1-Score: 0.98 - This high F1-score indicates strong performance for this class.
Class 1.0 (Sepsis):
Precision: 0.13 - When the model predicts 1.0 (Sepsis), it is correct only 13% of the time. This low precision suggests a high false-positive rate for the Sepsis class.
Recall: 0.14 - The model identifies only 14% of actual Sepsis cases, which is low and indicates that the model is failing to capture many true Sepsis cases.
F1-Score: 0.13 - The low F1-score suggests poor performance for class 1.0.
Macro and Weighted Averages:

Macro Avg (Unweighted): 0.56 for precision, recall, and F1-score – reflects an average across classes without accounting for their support, revealing the model’s struggle with minority class predictions.
Weighted Avg: 0.97 for all metrics – shows an average weighted by class support, so it’s heavily influenced by class 0.0 due to the imbalance.
Confusion Matrix:

_True Negatives (210,317) and False Positives (3,749) for class 0.0 indicate that the model largely predicts non-Sepsis correctly, with some false positives. True Positives (541) and False Negatives (3,378) for class 1.0 highlight the poor recall for Sepsis, as most true Sepsis cases are missed._

**Summary:**

The model’s performance on the validation set is skewed by the class imbalance: **It performs well on the majority class (non-Sepsis) but fails to effectively detect the minority class (Sepsis).**
This indicates that the model may benefit from techniques to handle class imbalance, such as:
Resampling techniques (e.g., oversampling the minority class or undersampling the majority class),
Class weights adjustment during training to penalize misclassification of the minority class,
Alternative evaluation metrics like the F1-score for the minority class or the AUC-ROC score, which can better reflect model performance in imbalanced datasets.
In summary, while the overall accuracy is high, the model has poor sensitivity for detecting Sepsis cases, making it unreliable for applications where identifying true positives in the minority class is critical.


In [16]:
from sklearn.tree import DecisionTreeClassifier

In [17]:
# Create a Decision Tree classifier

dt_model = DecisionTreeClassifier(random_state=272)

# Fit the model on the training data

dt_model.fit(X_train_syn, y_train_syn)

# Evaluate the Model

y_pred = dt_model.predict(X_train_syn)

# Calculate training accuracy

train_accuracy = accuracy_score(y_train_syn, y_train_pred)

print(f"Training Accuracy: {train_accuracy}")

# Detailed classification report

print("Classification Report on Training Set:")

print(classification_report(y_train_syn, y_train_pred))

# Confusion Matrix

print("Confusion Matrix on Training Set:")

print(confusion_matrix(y_train_syn, y_train_pred))

Training Accuracy: 0.9628159759406116
Classification Report on Training Set:
              precision    recall  f1-score   support

         0.0       0.95      0.98      0.96    852557
         1.0       0.97      0.95      0.96    852557

    accuracy                           0.96   1705114
   macro avg       0.96      0.96      0.96   1705114
weighted avg       0.96      0.96      0.96   1705114

Confusion Matrix on Training Set:
[[831509  21048]
 [ 42355 810202]]


In [18]:
# Check performance on validation set

# Predict on validation set

y_val_pred = dt_model.predict(X_val)

# Calculate Validation accuracy

val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Validation Accuracy: {val_accuracy}")

# Detailed classification report

print("Classification Report on Validation Set:")

print(classification_report(y_val, y_val_pred))

# Confusion Matrix

print("Confusion Matrix on Validation Set:")

print(confusion_matrix(y_val, y_val_pred))

Validation Accuracy: 0.9515162536940214
Classification Report on Validation Set:
              precision    recall  f1-score   support

         0.0       0.98      0.97      0.98    216018
         1.0       0.06      0.11      0.07      3932

    accuracy                           0.95    219950
   macro avg       0.52      0.54      0.52    219950
weighted avg       0.97      0.95      0.96    219950

Confusion Matrix on Validation Set:
[[208860   7158]
 [  3506    426]]
